In [ ]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Leer el archivo CSV de titulados ed_sup 2022
part1 = pd.read_csv('part1.csv')
part2 = pd.read_csv('part2.csv')
part3 = pd.read_csv('part3.csv')
part4 = pd.read_csv('part4.csv')
df_titulados = pd.concat([part1, part2, part3, part4])

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_titulados.head()

In [ ]:
df_notas2015 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2015_WEB.csv',sep=";", encoding='latin1')
df_notas2016 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2016_WEB.csv',sep=";", encoding='latin1')
df_notas2017 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2017_WEB.csv',sep=";", encoding='latin1')
df_notas2018 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2018_WEB.csv',sep=";", encoding='latin1')
df_notas2019 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2019_WEB.csv',sep=";", encoding='latin1')
df_notas2020 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2020_WEB.csv',sep=";", encoding='latin1')
df_notas2015_2020 = pd.concat([df_notas2015, df_notas2016, df_notas2017, df_notas2018, df_notas2019, df_notas2020])


In [ ]:
df_notas2015_2020.head()

Preprocesado titulados

In [ ]:
#vemos el tamaño para saber las dimensiones en futuras referencias
df_titulados.shape

In [ ]:
#evaluamos las columnas para ver que utilizaremos
df_titulados.head(5)

In [ ]:
#decidimos eliminar las columnas que no vemos util
columnas_a_eliminar = ['fec_nac_alu','anio_ing_carr_ori','sem_ing_carr_ori','sem_ing_carr_act','fecha_obtencion_titulo','cod_inst','cod_sede','cod_carrera','dur_proceso_tit','provincia_sede','comuna_sede','version','tipo_plan_carr','area_cineunesco','area_cine_f_97','subarea_cine_f_97','area_cine_f_13','subarea_cine_f_13','area_carrera_generica_n']
df_titulados.drop(columns=columnas_a_eliminar, inplace=True)

In [ ]:
#evaluamos si hay nulos
df_titulados.isnull().sum()

In [ ]:
#Notamos que mrun tiene nulos lo cual no lo consideraremos ya que son necesarios para un merge a futuro que haremos
df_titulados.dropna(subset=["mrun"], inplace=True)

In [ ]:
#tambien notamos que hay muchos nulos en nombres de itulos obtenidos, lo que asumimos que no lo consiguieron.
# por lo que lo rellenaremos como no completados
df_titulados["nomb_titulo_obtenido"].fillna('No completado', inplace=True)

In [ ]:
#lo mismo para el nombre de grado obtenido
df_titulados["nomb_grado_obtenido"].fillna('Grado superior no obtenido', inplace=True)

In [ ]:
#comprobamos si hay duplicados
df_titulados.duplicated().sum()
df_titulados.drop_duplicates

In [ ]:
#ahora vemos si los tippos de datos son los correctos en las columnas y si son necesarios cambiarlos 
df_titulados.info()

In [ ]:
#necesitamos el mrun como int para el merge

# Asegurarse de que sea de tipo int64
df_titulados['mrun'] = df_titulados['mrun'].astype(np.int64)

# Verificar el tipo de datos de la columna
print("verificamos si es de tipo int64:", df_titulados['mrun'].dtype)


Preprocesado notas

In [ ]:
df_notas2015_2020.shape

In [ ]:
df_notas2015_2020.head(2)

In [ ]:
#decidimos eliminar las columnas que no vemos util
columnas_a_eliminar = ['DGV_RBD','COD_REG_RBD','RBD','NOM_REG_RBD_A','COD_PRO_RBD','COD_COM_RBD','NOM_COM_RBD','COD_DEPROV_RBD','NOM_DEPROV_RBD']
df_notas2015_2020.drop(columns=columnas_a_eliminar, inplace=True)

In [ ]:
#evaluamos si hay nulos
df_notas2015_2020.isnull().sum()

In [ ]:
#comprobamos si hay duplicados
df_notas2015_2020.duplicated().sum()

In [ ]:
#ahora vemos si los tippos de datos son los correctos en las columnas y si son necesarios cambiarlos 
# (veremos en el codigo mas adelante si son necesarios)
df_notas2015_2020.info()

Pensando en el merge

In [ ]:
df_merge_titulados_notas=pd.merge(df_notas2015_2020,df_titulados, left_on="MRUN",right_on="mrun")

In [ ]:
df_merge_titulados_notas.shape

In [ ]:
df_merge_titulados_notas.head(5)

In [ ]:

#decidimos eliminar las columnas que no vemos util
columnas_a_eliminar = ['COD_GRADO','MRUN','codigo_unico','mrun','rango_edad','nomb_titulo_obtenido','tipo_inst_1','tipo_inst_2','tipo_inst_3','nomb_inst','nomb_sede','nomb_carrera','nivel_global','dur_estudio_carr','dur_total_carr','region_sede','jornada','modalidad','ORIGEN','nivel_carrera_1','nomb_grado_obtenido']
df_merge_titulados_notas.drop(columns=columnas_a_eliminar, inplace=True)


In [ ]:
# Eliminar las filas donde ENSE_COMPLETA es 0 asegurandonos de tratar con casos de egresados
df_merge_titulados_notas.drop(df_merge_titulados_notas[df_merge_titulados_notas['ENSE_COMPLETA'] == 0].index, inplace=True)

# Verificar el resultado
df_merge_titulados_notas[df_merge_titulados_notas["ENSE_COMPLETA"]==0]


In [ ]:
df_merge_titulados_notas.isnull().sum()

In [ ]:
df_merge_titulados_notas.duplicated().sum()

In [ ]:
df_merge_titulados_notas.drop_duplicates(inplace=True)

In [ ]:
#tener ojo con los tipos de datos para futuras referencias de tipos de datos
df_merge_titulados_notas.info()

In [ ]:
# Reemplazar comas por puntos
df_merge_titulados_notas['PROM_NOTAS_ALU'] = df_merge_titulados_notas['PROM_NOTAS_ALU'].str.replace(',', '.')

# Asegurarse de que la columna sea de tipo float64
df_merge_titulados_notas['PROM_NOTAS_ALU'] = df_merge_titulados_notas['PROM_NOTAS_ALU'].astype(np.float64)

# Verificar el tipo de datos de la columna
print("verificamos si es de tipo float64:", df_merge_titulados_notas['PROM_NOTAS_ALU'].dtype)

Inicio modelos de clasificacion

In [ ]:
df_merge_titulados_notas.head(5)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

# Supongamos que ya tienes el DataFrame cargado como df_merge_titulados_notas
# df_merge_titulados_notas = pd.read_csv('ruta_al_archivo.csv')  # Si necesitas cargar el archivo

# Filtrar el DataFrame para eliminar las filas donde ENSE_COMPLETA es 0
df_merge_titulados_notas.drop(df_merge_titulados_notas[df_merge_titulados_notas['ENSE_COMPLETA'] == 0].index, inplace=True)

# Seleccionar características relevantes
features = ['PROM_NOTAS_ALU', 'COD_ENSE', 'cat_periodo']

# Filtrar el DataFrame para obtener solo las características y la etiqueta
df = df_merge_titulados_notas[features + ['nivel_carrera_2']]

# Separar características y etiqueta
X = df[features]  # Aquí se asegura de que todas las características sean usadas
y = df['nivel_carrera_2']

# Dividir el conjunto de datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo: Árbol de Decisión
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
print(f'Precisión del modelo de Árbol de Decisión: {accuracy_tree:.2f}')

# Visualizar la matriz de confusión para el modelo de Árbol de Decisión
cm_tree = confusion_matrix(y_test, y_pred_tree, labels=tree_clf.classes_)
disp_tree = ConfusionMatrixDisplay(confusion_matrix=cm_tree, display_labels=tree_clf.classes_)
disp_tree.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusión para el Modelo de Árbol de Decisión')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el conjunto de datos de iris
iris = load_iris()

# Crear un DataFrame a partir de los datos de iris
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Dividir las características y la variable objetivo
X = df[iris.feature_names]
y = df['target']

# Dividir el conjunto de datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir con el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy:.2f}')

# Visualizar las predicciones con una matriz de confusión
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris.target_names)
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusión para las Predicciones del Modelo Random Forest')
plt.show()

# Visualizar las predicciones con gráficos de dispersión
plt.figure(figsize=(12, 6))
for i, feature in enumerate(iris.feature_names, 1):
    plt.subplot(2, 2, i)
    sns.scatterplot(x=X_test[feature], y=y_test, label='Real', marker='o')
    sns.scatterplot(x=X_test[feature], y=y_pred, label='Predicho', marker='x')
    plt.xlabel(feature)
    plt.ylabel('Target')
    plt.title(f'Predicción vs Realidad en {feature}')
    plt.legend()

plt.tight_layout()
plt.show()
